# Drop and splash centering


## Documentation and Imports


Created on 27-05-2021

Author: Valentin Laplaud


In [ ]:
# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2
mpl.rcParams['axes.facecolor'] = COLOR2

from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
from cycler import cycler
#Default colors
colorcycle = [plt.get_cmap('gist_rainbow')(1. * i/30) for i in range(30)]
mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)

# numbers handling
import numpy as np
import pandas as pd
    

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags, find_peaks
from scipy.interpolate import interp1d
from scipy import optimize

# images handling
from skimage import io
from skimage.filters import threshold_otsu, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray
from skimage.segmentation import active_contour
import cv2 as cv

# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../PythonFuncs/')
from Main_DP import splashShapeDyn, DropCentering, splashType, CropAndSave
import VallapFunc_DP as vf

print('Importations done.')

##  Enter data and run

### Data details

In [ ]:
###### Experiment of Ana to test analysis

# Spatial scale (px/mm)
Scale = 11.20 # To correct ???

# Frames per second
FPS = 5000 

# Paths to data
LoadP = r'd:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\DataAna'
SaveP = r'd:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\DataAna\SavedStuff'
# List of videos to analyze
VideoList = ['Today_Expe1','Today_Expe2']

print('Data choice made.')

### Run 

#### Croping and saving video as stack

In [ ]:

CropAndSave(VideoList,LoadP,SaveP,30, debug = False)


#### Drop centering

In [ ]:
# Run
ImageNumber = [1,6]
SplashData = DropCentering(SaveP,VideoList,Scale, ImageNumber, debug=True)

# Saving
SplashData.to_csv(SaveP + '\\SplashData_Cent.csv',index_label = 'Ind')

#### Splash type

In [ ]:
# Loading
SplashData = pd.read_csv(SaveP + '\\SplashData_Cent.csv',index_col = 'Ind')

# Selecting splash type
ImageNumbers = [20, 25, 30]
SplashData = splashType(SplashData,SaveP, ImageNumbers)

# Saving
SplashData.to_csv(SaveP + '\\SplashData_Cent_ST.csv',index_label = 'Ind')

## Test Zone